In [1]:
from forward import *
from util import *
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
from torch.optim import SGD

In [2]:
torch.manual_seed(1234)
train_loader, test_loader = MNIST_loaders()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




In [3]:
x, y = next(iter(train_loader))
x, y = x.to(device), y.to(device)
x_pos = overlay_y_on_x(x, y)
rnd = torch.randperm(x.size(0))
x_neg = overlay_y_on_x(x, y[rnd])

dataset = FwFw_Dataset(x_pos, x_neg)
train_dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

#for data, name in zip([x, x_pos, x_neg], ['orig', 'pos', 'neg']):
#    visualize_sample(data, name)


In [4]:

net = FCNet([784, 2000, 2000, 2000, 2000])

opt = SGD(net.parameters(), lr=0.005, momentum=0.9)
n_epoch = 100
lossfn = fwfw_loss
pbar = tqdm(range(n_epoch), desc=f"Epoch 0")
for i in pbar:
    running_loss = 0
    for batch in train_dataloader:
        cur_x = batch[0]
        cur_y = batch[1]
        opt.zero_grad()
        res = net.forward(cur_x)
        loss = lossfn(cur_y.type(torch.float64), res)
        loss.backward()
        opt.step()
        running_loss += (loss.item() / len(dataset))
    if i % 10 == 0:
        with torch.no_grad():
            acc = net.predict(x).eq(y).float().mean().item()
    pbar.set_description(f'Epoch {i}, train loss {running_loss}, train acc: {acc}')


Epoch 0:   0%|                                                                                                                                                                                                  | 0/100 [00:54<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
x_te, y_te = next(iter(test_loader))
x_te, y_te = x_te.to(device), y_te.to(device)
with torch.no_grad():
    print('train error:', 1.0 - net.predict(x).eq(y).float().mean().item())
    print('test error:', 1.0 - net.predict(x_te).eq(y_te).float().mean().item())


In [ ]:
#asdf = nn.BCEWithLogitsLoss()

#test_y = torch.tensor([0, -1, 1]).type(torch.float64)
#test_logits = torch.tensor([-90, 99, 999]).type(torch.float64)
#print(asdf(test_logits, test_y))